In [1]:
### imports

# external modules
import os
import sys
import json
import time
import numpy as np
from fnmatch import fnmatch
import importlib

# local modules
thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../../'))
sys.path.append(topdir)

import tools.iotools as iotools
import tools.dftools as dftools
import tools.omstools as omstools
from tools.omsapi.get_oms_data import get_oms_api
from tools.omsapi.get_oms_data import get_oms_data
from tools.omsapi.get_oms_data import get_oms_response_attribute
from tools.omsapi.get_oms_data import get_oms_response_attributes
from tools.omsapi.examples import get_lumisection_info
from tools.omsapi.examples import get_hltrate_info
from tools.omsapi.examples import get_hltprescale_info

In [2]:
# get the omsapi instance

omsapi = get_oms_api()

In [3]:
# define run ranges

# references:
# - https://twiki.cern.ch/twiki/bin/view/CMS/PdmVRun3Analysis#2024_Era_definition
# - https://twiki.cern.ch/twiki/bin/view/CMS/PdmVRun3Analysis#2025_Era_definition

# note: there seems to be a discrepancy between the twiki above and the runs in the DQMIO files...
# see e.g. check_data_consistency.ipynb

#eradict = { # copied from the twiki
#  'Run2024A-v1': (378142, 378970),
#  'Run2024B-v1': (378971, 379411),
#  'Run2024C-v1': (379412, 380252),
#  'Run2024D-v1': (380253, 380947),
#  'Run2024E-v1': (380948, 381383),
#  'Run2024E-v2': (381384, 381943),
#  'Run2024F-v1': (381944, 383779),
#  'Run2024G-v1': (383780, 385813),
#  'Run2024H-v1': (385814, 386408),
#  'Run2024I-v1': (386409, 386797),
#  'Run2024I-v2': (386798, 387121),
#  'Run2024J-v1': (387203, 387721)
#}

eradict = { # modified according to the run numbers observed in the DQMIO files
  #'Run2024A-v1': (378142, 378970),
  #'Run2024B-v1': (378971, 379411),
  #'Run2024C-v1': (379412, 380252),
  #'Run2024D-v1': (380253, 380947),
  #'Run2024E-v1': (380948, 381383),
  #'Run2024E-v2': (381384, 381943),
  #'Run2024F-v1': (381944, 383779),
  #'Run2024G-v1': (383780, 385813),
  #'Run2024H-v1': (385814, 386408),
  #'Run2024I-v1': (386409, 386693), # modified here
  #'Run2024I-v2': (386694, 387121), # modified here
  #'Run2024J-v1': (387203, 387721),
  #'Run2025B-v1': (391531, 392158),
  #'Run2025C-v1': (392159, 393108),
  #'Run2025C-v2': (393111, 393609),
  #'Run2025D-v1': (394286, 395967),
  #'Run2025E-v1': (395968, 396597),
  'Run2025F-v1': (396598, 397607), # temporary, last run currently in DAS, but era still ongoing.
}

In [5]:
# find runs that are present in the DQMIO data (for filtering the OMS response)

# define settings for which files to retrieve the run numbers from
datadir = '/eos/user/l/llambrec/dialstools-output'
dataset = 'ZeroBias'
reco = 'PromptReco'
me = 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_3'

# find files corresponding to settings
filepattern = f'{dataset}-Run*-{reco}-*-DQMIO-{me}.parquet'
files = sorted([os.path.join(datadir, f) for f in os.listdir(datadir) if fnmatch(f, filepattern)])

# retrieve run numbers
dqmio_runs = []
print(f'Reading {len(files)} files for retrieving run numbers...')
for f in files:
    df = iotools.read_parquet(f, columns=['run_number'])
    run_numbers = dftools.get_runs(df, runcolumn='run_number')
    dqmio_runs += run_numbers
dqmio_runs = sorted(list(set(dqmio_runs)))
dqmio_runs = np.array(dqmio_runs).astype(int)
print(f'Found {len(dqmio_runs)} runs.')

Reading 26 files for retrieving run numbers...
Found 6033 runs.


**Part 1: pileup, luminosity, and similar attributes per lumisection**

This does not include HLT trigger rates, which are retrieved separately in part 2 further down.

In [6]:
# find out which attributes are available per lumisection

runnb = 378142 # dummy run, should not matter
ls_info = get_oms_data( omsapi, 'lumisections', runnb )
available_attributes = sorted(list(get_oms_response_attributes(ls_info)))
print(available_attributes)

https://cmsoms.cern.ch/agg/api/v1/lumisections?filter[run_number][EQ]=378142&page[offset]=0&page[limit]=1000
['beam1_present', 'beam1_stable', 'beam2_present', 'beam2_stable', 'beam_present', 'beams_stable', 'bpix_ready', 'castor_ready', 'cms_active', 'cscm_ready', 'cscp_ready', 'delivered_lumi', 'delivered_lumi_per_lumisection', 'dt0_ready', 'dtm_ready', 'dtp_ready', 'ebm_ready', 'ebp_ready', 'eem_ready', 'eep_ready', 'end_lumi', 'end_time', 'esm_ready', 'esp_ready', 'fill_number', 'fpix_ready', 'gem_ready', 'gemm_ready', 'gemp_ready', 'hbhea_ready', 'hbheb_ready', 'hbhec_ready', 'hf_ready', 'ho_ready', 'init_lumi', 'lumisection_number', 'physics_flag', 'pileup', 'prescale_index', 'prescale_name', 'recorded_lumi', 'recorded_lumi_per_lumisection', 'rp_sect_45_ready', 'rp_sect_56_ready', 'rp_time_ready', 'rpc_ready', 'run_number', 'start_time', 'tecm_ready', 'tecp_ready', 'tibtid_ready', 'tob_ready', 'zdc_ready']


In [7]:
# define attributes to retrieve

attributes = [
    'delivered_lumi_per_lumisection',
    'recorded_lumi_per_lumisection',
    'pileup',
    'physics_flag',
    'prescale_index',
    'prescale_name',
    'beams_stable',
    'cms_active',
    'bpix_ready',
    'fpix_ready',
    'tibtid_ready',
    'tob_ready',
    'tecp_ready',
    'tecm_ready',
    'fill_number',
    'run_number',
    'lumisection_number',
]

# quick check
for attribute in attributes:
    if attribute not in available_attributes:
        print(f'WARNING: attribute {attribute} does not seem to be available.')

In [8]:
# get the data from OMS

defaults = {
    'physics_flag': False,
    'pileup': 0,
    'delivered_lumi_per_lumisection': 0,
    'recorded_lumi_per_lumisection': 0,
    'prescale_index': -1,
    'prescale_name': '',
    'beams_stable': False,
    'cms_active': False,
    'bpix_ready': False,
    'fpix_ready': False,
    'tibtid_ready': False,
    'tob_ready': False,
    'tecp_ready': False,
    'tecm_ready': False
}

info = {}
for era, runrange in eradict.items():
    print('Retrieving data for era {}'.format(era))
    ls_info = get_lumisection_info(omsapi, runrange, attributes, defaults=defaults, run_filter=dqmio_runs)
    info[era] = ls_info

Retrieving data for era Run2025F-v1
Making API call...
https://cmsoms.cern.ch/agg/api/v1/lumisections?fields=lumisection_number&filter[run_number][GE]=396598&filter[run_number][LE]=397607&page[offset]=0&page[limit]=100000
https://cmsoms.cern.ch/agg/api/v1/lumisections?fields=tecm_ready,delivered_lumi_per_lumisection,beams_stable,prescale_name,tibtid_ready,physics_flag,fpix_ready,bpix_ready,recorded_lumi_per_lumisection,pileup,lumisection_number,cms_active,run_number,fill_number,tecp_ready,prescale_index,tob_ready&filter[run_number][GE]=396598&filter[run_number][LE]=397607&page[offset]=0&page[limit]=100000
Found 88935 lumisections with 17 attributes
Substituting None values with defaults...
  - Now checking attribute physics_flag...
  - Now checking attribute pileup...
  - Now checking attribute delivered_lumi_per_lumisection...
  - Now checking attribute recorded_lumi_per_lumisection...
  - Now checking attribute prescale_index...
    Found 10206 None instances (out of 88935 total); su

In [9]:
# save to json files

outputdir = 'omsdata'
if not os.path.exists(outputdir): os.makedirs(outputdir)
    
for era in eradict.keys():
    thisinfo = info[era]
    outputfile = os.path.join(outputdir, 'omsdata_{}.json'.format(era))
    with open(outputfile, 'w') as f:
        json.dump(thisinfo, f)
    print(f'Created file {outputfile}')

Created file omsdata/omsdata_Run2025F-v1.json


**Part 2: HLT trigger rates**

In [10]:
# get the HLT rate from OMS

trigger_patterns = [
    'HLT_Physics_v*',
    'HLT_ZeroBias_v*',
    'DQM_PixelReconstruction_v*'
]

info = {}
for era, runrange in eradict.items():
    print(f'Retrieving data for era {era}')
    outputdir = f'omsdata_temp_{era}'
    outputfilebase = f'hltrate_{era}'
    rate_info = get_hltrate_info(omsapi, runrange, trigger_patterns,
                     run_filter=dqmio_runs, outputdir=outputdir, outputfilebase=outputfilebase)
    info[era] = rate_info

Retrieving data for era Run2025F-v1
https://cmsoms.cern.ch/agg/api/v1/runs?fields=run_number&filter[run_number][GE]=396598&filter[run_number][LE]=397607&page[offset]=0&page[limit]=2000
Found 662 runs in OMS, of which 504 are in the DQMIO files.
Looping over 504 runs...
https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[run_number][EQ]=396600&page[offset]=0&page[limit]=10000
Run 396600: found following triggers: ['DQM_PixelReconstruction_v14', 'HLT_Physics_v15'].
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396600&filter[path_name][EQ]=DQM_PixelReconstruction_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396600&filter[path_name][EQ]=HLT_Physics_v15&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fi

https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396629&filter[path_name][EQ]=HLT_ZeroBias_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[run_number][EQ]=396630&page[offset]=0&page[limit]=10000
Run 396630: found following triggers: ['DQM_PixelReconstruction_v14', 'HLT_Physics_v15'].
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396630&filter[path_name][EQ]=DQM_PixelReconstruction_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396630&filter[path_name][EQ]=HLT_Physics_v15&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[run_number][EQ]=3966

https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[run_number][EQ]=396697&page[offset]=0&page[limit]=10000
Run 396697: found following triggers: ['DQM_PixelReconstruction_v14', 'HLT_Physics_v15', 'HLT_ZeroBias_v14'].
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396697&filter[path_name][EQ]=DQM_PixelReconstruction_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396697&filter[path_name][EQ]=HLT_Physics_v15&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396697&filter[path_name][EQ]=HLT_ZeroBias_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[

https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[run_number][EQ]=396708&page[offset]=0&page[limit]=10000
Run 396708: found following triggers: ['DQM_PixelReconstruction_v14', 'HLT_Physics_v15', 'HLT_ZeroBias_v14'].
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396708&filter[path_name][EQ]=DQM_PixelReconstruction_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396708&filter[path_name][EQ]=HLT_Physics_v15&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396708&filter[path_name][EQ]=HLT_ZeroBias_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[

https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[run_number][EQ]=396726&page[offset]=0&page[limit]=10000
Run 396726: found following triggers: ['DQM_PixelReconstruction_v14', 'HLT_Physics_v15', 'HLT_ZeroBias_v14'].
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396726&filter[path_name][EQ]=DQM_PixelReconstruction_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396726&filter[path_name][EQ]=HLT_Physics_v15&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=396726&filter[path_name][EQ]=HLT_ZeroBias_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[

...

https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=397501&filter[path_name][EQ]=HLT_Physics_v15&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=397501&filter[path_name][EQ]=HLT_ZeroBias_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[run_number][EQ]=397502&page[offset]=0&page[limit]=10000
Run 397502: found following triggers: ['DQM_PixelReconstruction_v14', 'HLT_Physics_v15', 'HLT_ZeroBias_v14'].
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=397502&filter[path_name][EQ]=DQM_PixelReconstruction_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first

https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=397528&filter[path_name][EQ]=HLT_Physics_v15&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=397528&filter[path_name][EQ]=HLT_ZeroBias_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[run_number][EQ]=397529&page[offset]=0&page[limit]=10000
Run 397529: found following triggers: ['DQM_PixelReconstruction_v14', 'HLT_Physics_v15', 'HLT_ZeroBias_v14'].
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=397529&filter[path_name][EQ]=DQM_PixelReconstruction_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first

https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[run_number][EQ]=397545&page[offset]=0&page[limit]=10000
Run 397545: found following triggers: ['DQM_PixelReconstruction_v14', 'HLT_Physics_v15'].
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=397545&filter[path_name][EQ]=DQM_PixelReconstruction_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=397545&filter[path_name][EQ]=HLT_Physics_v15&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[run_number][EQ]=397546&page[offset]=0&page[limit]=10000
Run 397546: found following triggers: ['DQM_PixelReconstruction_v14', 'HLT_Physics_v15'].
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[

https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=397568&filter[path_name][EQ]=HLT_Physics_v15&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=397568&filter[path_name][EQ]=HLT_ZeroBias_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[run_number][EQ]=397569&page[offset]=0&page[limit]=10000
Run 397569: found following triggers: ['DQM_PixelReconstruction_v14', 'HLT_Physics_v15', 'HLT_ZeroBias_v14'].
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=397569&filter[path_name][EQ]=DQM_PixelReconstruction_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first

https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[run_number][EQ]=397588&page[offset]=0&page[limit]=10000
Run 397588: found following triggers: ['DQM_PixelReconstruction_v14', 'HLT_Physics_v15'].
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=397588&filter[path_name][EQ]=DQM_PixelReconstruction_v14&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[run_number][EQ]=397588&filter[path_name][EQ]=HLT_Physics_v15&page[offset]=0&page[limit]=10000&group[granularity]=lumisection
https://cmsoms.cern.ch/agg/api/v1/hltpathinfo?fields=path_name&filter[run_number][EQ]=397590&page[offset]=0&page[limit]=10000
Run 397590: found following triggers: ['DQM_PixelReconstruction_v14', 'HLT_Physics_v15'].
https://cmsoms.cern.ch/agg/api/v1/hltpathrates?fields=run_number,first_lumisection_number,rate&filter[

In [11]:
# save to json files

outputdir = 'omsdata_test'
if not os.path.exists(outputdir): os.makedirs(outputdir)
    
for era in eradict.keys():
    thisinfo = info[era]
    outputfile = os.path.join(outputdir, 'hltrate_{}.json'.format(era))
    with open(outputfile, 'w') as f:
        json.dump(thisinfo, f)
    print(f'Created file {outputfile}')

Created file omsdata_test/hltrate_Run2025F-v1.json


In [ ]:
# alternative approach:
# combine the per-run json files into per-era files

outputdir = 'omsdata_test2'
if not os.path.exists(outputdir): os.makedirs(outputdir)
    
for era, runrange in eradict.items():
    # set input and output files
    inputdir = f'omsdata_temp_{era}'
    inputfiles = sorted([os.path.join(inputdir, f) for f in os.listdir(inputdir)
                   if fnmatch(f, f'hltrate_{era}_??????.json')])
    if len(inputfiles)==0:
        print(f'WARNING: no input files found for era {era}, skipping.')
        continue
    outputfile = os.path.join(outputdir, f'hltrate_{era}.json')
    hltrates = {}
    for inputfile in inputfiles:
        run = int(inputfile.split('_')[-1].replace('.json', ''))
        with open(inputfile, 'r') as f:
            info = json.load(f)
        hltrates[run] = info
    with open(outputfile, 'w') as f:
        json.dump(hltrates, f)
    print(f'Merged {len(inputfiles)} into {outputfile}.')

**Part 3: trigger prescales**

In [ ]:
# get the HLT prescale info from OMS

trigger_patterns = [
    'HLT_Physics_v*',
    'HLT_ZeroBias_v*',
    'DQM_PixelReconstruction_v*'
]

import tools.omsapi.examples as temp
importlib.reload(temp)
from tools.omsapi.examples import get_hltprescale_info

info = {}
for era, runrange in eradict.items():
    print('Retrieving data for era {}'.format(era))
    outputdir = f'omsdata_temp_{era}'
    outputfilebase = f'hltprescale_{era}'
    prescale_info = get_hltprescale_info(omsapi, runrange, trigger_patterns,
                     run_filter=dqmio_runs, outputdir=outputdir, outputfilebase=outputfilebase)
    info[era] = prescale_info

In [ ]:
# save to json files

outputdir = 'omsdata_test'
if not os.path.exists(outputdir): os.makedirs(outputdir)
    
for era in eradict.keys():
    thisinfo = info[era]
    outputfile = os.path.join(outputdir, 'hltprescale_{}.json'.format(era))
    with open(outputfile, 'w') as f:
        json.dump(thisinfo, f)
    print(f'Created file {outputfile}')

In [ ]:
# alternative approach:
# combine the per-run json files into per-era files

outputdir = 'omsdata_test2'
if not os.path.exists(outputdir): os.makedirs(outputdir)
    
for era, runrange in eradict.items():
    # set input and output files
    inputdir = f'omsdata_temp_{era}'
    inputfiles = sorted([os.path.join(inputdir, f) for f in os.listdir(inputdir)
                   if fnmatch(f, f'hltprescale_{era}_??????.json')])
    if len(inputfiles)==0:
        print(f'WARNING: no input files found for era {era}, skipping.')
        continue
    outputfile = os.path.join(outputdir, f'hltprescale_{era}.json')
    hltrates = {}
    for inputfile in inputfiles:
        run = int(inputfile.split('_')[-1].replace('.json', ''))
        with open(inputfile, 'r') as f:
            info = json.load(f)
        hltrates[run] = info
    with open(outputfile, 'w') as f:
        json.dump(hltrates, f)
    print(f'Merged {len(inputfiles)} into {outputfile}.')

In [ ]:
# post-processing: make files with per-lumisection prescale values

outputdir = 'omsdata_test'
if not os.path.exists(outputdir): os.makedirs(outputdir)

def find_prescale(prescale_info, prescale_index, prescale_name=None):
    match = None
    prescale_index = int(prescale_index)
    if prescale_index < 0: return 0
    for el in prescale_info:
        if int(el['prescale_index']) != prescale_index: continue
        if prescale_name is not None and el['prescale_name'] != prescale_name: continue
        match = el
    prescale = match['prescale']
    return prescale

for era in eradict.keys():
    print(f'Processing era {era}...')
    
    # load OMS data
    omsfile = os.path.join(outputdir, f'omsdata_{era}.json')
    with open(omsfile, 'r') as f:
        omsdata = json.load(f)
    
    # load trigger rate data
    ratefile = os.path.join(outputdir, f'hltrate_{era}.json')
    with open(ratefile, 'r') as f:
        ratedata = json.load(f)
    
    # load prescale data
    prescalefile = os.path.join(outputdir, f'hltprescale_{era}.json')
    with open(prescalefile, 'r') as f:
        prescaledata = json.load(f)
        
    # loop over runs and triggers
    prescales = {}
    for run in ratedata.keys():
        prescales[run] = {}
        for trigger in ratedata[run].keys():
            # get rate info for this run and trigger
            rate_info = ratedata[run][trigger]
            this_lumis = np.array(rate_info['first_lumisection_number'])
            this_runs = np.array(rate_info['run_number'])
            # get prescale info for this run and trigger
            prescale_info = prescaledata[run][trigger]
            # get prescale index and name per lumisection for this run and trigger
            this_prescale_index = omstools.find_oms_attr_for_lumisections(this_runs, this_lumis, omsdata, 'prescale_index')
            this_prescale_index = [int(el) for el in this_prescale_index]
            nnone = this_prescale_index.count(None)
            if nnone!=0: 
                print('Found {} None instances (out of {} total) for era {}'.format(nnone, len(thisinfo), era))
                this_prescale_index = [el if el is not None else -1 for el in this_prescale_index]
            # find prescale from prescale index
            this_prescale = [find_prescale(prescale_info, idx) for idx in this_prescale_index]
            ratedata[run][trigger]['prescale'] = this_prescale
            ratedata[run][trigger]['prescale_index'] = this_prescale_index
            
    # write output file
    outputfile = ratefile.replace('.json', '_withprescale.json')
    with open(outputfile, 'w') as f:
        json.dump(ratedata, f)
    print(f'Created output file {outputfile}.')